In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np


In [9]:
basePath = '/content/drive/MyDrive/DATN/Code/Solution 2 - CIC-DDoS2019'

## tải mô hình resnet 50

Sử dụng mô hình ResNet50 đã được pre-trained trên ImageNet và loại bỏ lớp cuối cùng để thêm các lớp mới phù hợp với bài toán của bạn.

In [13]:
base_model = ResNet50(weights='imagenet', include_top=False) # Tải mô hình ResNet50 đã được huấn luyện trước trên ImageNet, bỏ qua các lớp fully connected cuối cùng


x = base_model.output # Lấy output từ mô hình gốc


x = GlobalAveragePooling2D()(x) # Thêm một lớp GlobalAveragePooling2D để giảm chiều dữ liệu


x = Dense(1024, activation='relu')(x) # Thêm một lớp Dense với 1024 neurons và hàm kích hoạt ReLU


num_classes = 8 # số lượng nhãn của dữ liệu
predictions = Dense(num_classes, activation='softmax')(x) # Thêm lớp đầu ra với số lượng neurons bằng số lớp của bài toán và hàm kích hoạt softmax

model = Model(inputs=base_model.input, outputs=predictions) # Tạo mô hình mới với đầu vào là đầu vào của mô hình gốc và đầu ra là lớp predictions

## Đóng băng các lớp của mô hình gốc

Để giữ lại các trọng số đã học từ ImageNet, bạn cần đóng băng các lớp của mô hình gốc

In [14]:
for layer in base_model.layers:
    layer.trainable = False


## Huấn luyện mô hình

Sử dụng các bộ dữ liệu đã chuẩn bị để huấn luyện mô hình

In [15]:
train_dir = basePath + '/datasets/CIC-DDoS2019/10 percent/train'
val_dir = basePath + '/datasets/CIC-DDoS2019/10 percent/val'

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
    shear_range=0.2,  # Áp dụng biến đổi shear
    zoom_range=0.2,  # Áp dụng biến đổi zoom
    horizontal_flip=True  # Lật ngang hình ảnh
)

validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Đường dẫn đến thư mục chứa dữ liệu huấn luyện
    target_size=(224, 224),  # Kích thước của hình ảnh đầu vào
    batch_size=32,  # Kích thước batch
    class_mode='categorical'  # Chế độ phân loại
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,  # Đường dẫn đến thư mục chứa dữ liệu xác thực
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Biên dịch mô hình
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train
model.fit(train_generator, epochs=10, validation_data=validation_generator)


Found 7510 images belonging to 8 classes.
Found 2503 images belonging to 8 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


235/235 ━━━━━━━━━━━━━━━━━━━━ 2017s 8s/step - accuracy: 0.5555 - loss: 1.1977 - val_accuracy: 0.9129 - val_loss: 0.4732
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1816s 8s/step - accuracy: 0.8882 - loss: 0.4747 - val_accuracy: 0.9604 - val_loss: 0.2091
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1794s 8s/step - accuracy: 0.9295 - loss: 0.2920 - val_accuracy: 0.9688 - val_loss: 0.1536
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1842s 8s/step - accuracy: 0.9520 - loss: 0.2001 - val_accuracy: 0.9600 - val_loss: 0.1464
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1787s 8s/step - accuracy: 0.9499 - loss: 0.1890 - val_accuracy: 0.9353 - val_loss: 0.2010
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1819s 8s/step - accuracy: 0.9527 - loss: 0.1646 - val_accuracy: 0.9628 - val_loss: 0.1095
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1776s 8s/step - accuracy: 0.9662 - loss: 0.1258 - val_accuracy: 0.9876 - val_loss: 0.0561
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1793s 8s/step - accuracy: 0.9703 - loss: 0.1095 - val_accurac

## Fine-tune các lớp cuối cùng:

Sau khi huấn luyện các lớp mới, bạn có thể mở khóa một số lớp cuối cùng của mô hình gốc để fine-tune.

In [20]:
# fine tune 4 lớp cuối

for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=2, validation_data=validation_generator)

Epoch 1/2
235/235 ━━━━━━━━━━━━━━━━━━━━ 1859s 8s/step - accuracy: 0.9275 - loss: 0.4351 - val_accuracy: 0.4379 - val_loss: 4.9009
Epoch 2/2
235/235 ━━━━━━━━━━━━━━━━━━━━ 1860s 8s/step - accuracy: 0.9636 - loss: 0.1200 - val_accuracy: 0.9305 - val_loss: 0.2259


In [39]:
# Lưu mô hình vào file 'my_model.h5'
model.save(basePath + '/models/fine_tuned_model.keras')

# Load model và dự đoán

In [2]:
# Tải lại mô hình từ file 'my_model.h5'
emodel = load_model('./models/fine_tuned_model.keras')

In [ ]:
emodel.summary()

### kiểm thử từng đầu vào

In [6]:
img_path = '/content/drive/MyDrive/DATN/Code/Solution 2 - CIC-DDoS2019/datasets/CIC-DDoS2019/10 percent/test/UDP/image_100221.png'

# Load và chuẩn hóa hình ảnh
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Thêm một chiều để phù hợp với đầu vào của mô hình
img_array /= 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]

In [7]:
# Dự đoán nhãn
predictions = emodel.predict(img_array)

# Lấy nhãn có xác suất cao nhất
predicted_class = np.argmax(predictions, axis=1)

print(f'Dự đoán nhãn: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Dự đoán nhãn: [0]


### kiểm thử trên tập test

In [54]:
# thử trên tập test

# Tạo đối tượng ImageDataGenerator cho tập test
test_datagen = ImageDataGenerator(rescale=1./255)

# Tạo generator cho tập test
test_generator = test_datagen.flow_from_directory(
    basePath + '/datasets/CIC-DDoS2019/10 percent/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 2504 images belonging to 8 classes.


In [55]:
# Đánh giá mô hình trên tập test
test_loss, test_accuracy = emodel.evaluate(test_generator)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


79/79 ━━━━━━━━━━━━━━━━━━━━ 513s 6s/step - accuracy: 0.9665 - loss: 0.1076
Test Loss: 0.20773868262767792
Test Accuracy: 0.9329073429107666
